<center>
<img src="../../img/ods_stickers.jpg">
    
## [mlcourse.ai](https://mlcourse.ai) - Open Machine Learning Course

Author: [Yury Kashnitsky](https://www.linkedin.com/in/festline/). All content is distributed under the [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/) license.

# <center> Assignment #6 (demo)
## <center>  Exploring OLS, Lasso and Random Forest in a regression task
    
**Same assignment as a [Kaggle Kernel](https://www.kaggle.com/kashnitsky/a6-demo-linear-models-and-rf-for-regression) + [solution](https://www.kaggle.com/kashnitsky/a6-demo-regression-solution).**    
    
<img src='../../img/wine_quality.jpg' width=30%>

**Fill in the missing code and choose answers in [this](https://docs.google.com/forms/d/1aHyK58W6oQmNaqEfvpLTpo6Cb0-ntnvJ18rZcvclkvw/edit) web form.**

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.metrics.regression import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
from sklearn.ensemble import RandomForestRegressor

**We are working with UCI Wine quality dataset (no need to download it – it's already there, in course repo and in Kaggle Dataset).**

In [2]:
data = pd.read_csv('../../data/winequality-white.csv', sep=';')

In [4]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


**Separate the target feature, split data in 7:3 proportion (30% form a holdout set, use random_state=17), and preprocess data with `StandardScaler`.**

In [10]:
y = data['quality']
X = data.drop(columns=['quality'])

X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, train_size=0.7)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)

## Linear regression

**Train a simple linear regression model (Ordinary Least Squares).**

In [11]:
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

**<font color='red'>Question 1:</font> What are mean squared errors of model predictions on train and holdout sets?**

In [12]:
print("Mean squared error (train): %.3f" % mean_squared_error(linreg.predict(X_train_scaled), y_train))
print("Mean squared error (test): %.3f" % mean_squared_error(linreg.predict(X_holdout_scaled), y_holdout))

Mean squared error (train): 0.561
Mean squared error (test): 0.576


**Sort features by their influence on the target feature (wine quality). Beware that both large positive and large negative coefficients mean large influence on target. It's handy to use `pandas.DataFrame` here.**

**<font color='red'>Question 2:</font> Which feature this linear regression model treats as the most influential on wine quality?**

In [20]:
coefs = pd.DataFrame({'name': X_train.columns, 'val':linreg.coef_, 'abs_val': np.abs(linreg.coef_)})
coefs.sort_values('abs_val', ascending=False)

,name,val,abs_val
7,density,-0.666728,0.666728
3,residual sugar,0.536700,0.536700
1,volatile acidity,-0.191059,0.191059
8,pH,0.134153,0.134153
10,alcohol,0.122436,0.122436
0,fixed acidity,0.096663,0.096663
9,sulphates,0.078061,0.078061
5,free sulfur dioxide,0.039480,0.039480
2,citric acid,0.015001,0.015001
4,chlorides,-0.003283,0.003283


## Lasso regression

**Train a LASSO model with $\alpha = 0.01$ (weak regularization) and scaled data. Again, set random_state=17.**

In [22]:
lasso1 = Lasso(alpha=1e-2, random_state=17)
lasso1.fit(X_train_scaled, y_train)

Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=17,
      selection='cyclic', tol=0.0001, warm_start=False)

**Which feature is the least informative in predicting wine quality, according to this LASSO model?**

In [24]:
lasso1_coef = pd.DataFrame({'name': X_train.columns, 'val':lasso1.coef_, 'abs_val': np.abs(lasso1.coef_)})
lasso1_coef.sort_values('abs_val', ascending=True)

,name,val,abs_val
0,fixed acidity,-0.000000,0.000000
2,citric acid,0.000000,0.000000
6,total sulfur dioxide,-0.000000,0.000000
4,chlorides,-0.012222,0.012222
5,free sulfur dioxide,0.032151,0.032151
9,sulphates,0.045397,0.045397
8,pH,0.046095,0.046095
1,volatile acidity,-0.193547,0.193547
7,density,-0.221868,0.221868
3,residual sugar,0.246254,0.246254


**Train LassoCV with random_state=17 to choose the best value of $\alpha$ in 5-fold cross-validation.**

In [25]:
alphas = np.logspace(-6, 2, 200)
lasso_cv = LassoCV(alphas=alphas, random_state=17)
lasso_cv.fit(X_train_scaled, y_train)

LassoCV(alphas=array([1.00000000e-06, 1.09698580e-06, 1.20337784e-06, 1.32008840e-06,
       1.44811823e-06, 1.58856513e-06, 1.74263339e-06, 1.91164408e-06,
       2.09704640e-06, 2.30043012e-06, 2.52353917e-06, 2.76828663e-06,
       3.03677112e-06, 3.33129479e-06, 3.65438307e-06, 4.00880633e-06,
       4.39760361e-06, 4.82410870e-06, 5.29197874e-06, 5.80522552e-06,
       6.36824994e-06, 6.98587975e-0...
       3.61234270e+01, 3.96268864e+01, 4.34701316e+01, 4.76861170e+01,
       5.23109931e+01, 5.73844165e+01, 6.29498899e+01, 6.90551352e+01,
       7.57525026e+01, 8.30994195e+01, 9.11588830e+01, 1.00000000e+02]),
        copy_X=True, cv=None, eps=0.001, fit_intercept=True, max_iter=1000,
        n_alphas=100, n_jobs=None, normalize=False, positive=False,
        precompute='auto', random_state=17, selection='cyclic', tol=0.0001,
        verbose=False)

In [30]:
lasso_cv.alpha_

0.0007149428986597577

**<font color='red'>Question 3:</font> Which feature is the least informative in predicting wine quality, according to the tuned LASSO model?**

In [31]:
lasso_cv_coef = pd.DataFrame({'name': X_train.columns, 'val':lasso_cv.coef_, 'abs_val': np.abs(lasso_cv.coef_)})
lasso_cv_coef.sort_values('abs_val', ascending=True)

,name,val,abs_val
6,total sulfur dioxide,-0.002739,0.002739
4,chlorides,-0.004269,0.004269
2,citric acid,0.014089,0.014089
5,free sulfur dioxide,0.039061,0.039061
9,sulphates,0.075298,0.075298
0,fixed acidity,0.086455,0.086455
8,pH,0.125789,0.125789
10,alcohol,0.141640,0.141640
1,volatile acidity,-0.191427,0.191427
3,residual sugar,0.510323,0.510323


**<font color='red'>Question 4:</font> What are mean squared errors of tuned LASSO predictions on train and holdout sets?**

In [32]:
print("Mean squared error (train): %.3f" % mean_squared_error(lasso_cv.predict(X_train_scaled), y_train))
print("Mean squared error (test): %.3f" % mean_squared_error(lasso_cv.predict(X_holdout_scaled), y_holdout))

Mean squared error (train): 0.561
Mean squared error (test): 0.574


## Random Forest

**Train a Random Forest with out-of-the-box parameters, setting only random_state to be 17.**

In [33]:
forest = RandomForestRegressor(random_state=17)
forest.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=17, verbose=0, warm_start=False)

**<font color='red'>Question 5:</font> What are mean squared errors of RF model on the training set, in cross-validation (cross_val_score with scoring='neg_mean_squared_error' and other arguments left with default values) and on holdout set?**

In [36]:
print("Mean squared error (train): %.3f" % mean_squared_error(forest.predict(X_train), y_train))
print("Mean squared error (cv): %.3f" % -np.mean(cross_val_score(forest, X_train, y_train, scoring='neg_mean_squared_error')))
print("Mean squared error (test): %.3f" % mean_squared_error(forest.predict(X_holdout), y_holdout))

Mean squared error (train): 0.053
Mean squared error (cv): 0.402
Mean squared error (test): 0.402


**Tune the `max_features` and `max_depth` hyperparameters with GridSearchCV and again check mean cross-validation MSE and MSE on holdout set.**

In [39]:
forest_params = {'max_depth': list(range(10, 25)), 
                 'min_samples_leaf': list(range(1, 8)),
                 'max_features': list(range(6,12))}

locally_best_forest = GridSearchCV(forest, forest_params, scoring='neg_mean_squared_error', n_jobs=-1, verbose=10)
locally_best_forest.fit(X_train, y_train)

Fitting 5 folds for each of 630 candidates, totalling 3150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    6.4s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:    8.0s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    9.3s
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:   11.7s
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:   14.5s
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:   17.3s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:   20.6s
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:   24.5s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   28.1s
[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed:   32.3s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   

GridSearchCV(cv=None, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=17,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_j

In [42]:
locally_best_forest.best_params_, locally_best_forest.best_score_

({'max_depth': 21, 'max_features': 6, 'min_samples_leaf': 1},
 -0.3936357900227976)

In [43]:
best_forrest = locally_best_forest.best_estimator_

**<font color='red'>Question 6:</font> What are mean squared errors of tuned RF model in cross-validation (cross_val_score with scoring='neg_mean_squared_error' and other arguments left with default values) and on holdout set?**

In [44]:
print("Mean squared error (cv): %.3f" % -np.mean(cross_val_score(best_forrest, X_train, y_train, scoring='neg_mean_squared_error')))
print("Mean squared error (test): %.3f" % mean_squared_error(best_forrest.predict(X_holdout), y_holdout))

Mean squared error (cv): 0.394
Mean squared error (test): 0.405


**Output RF's feature importance. Again, it's nice to present it as a DataFrame.**<br>
**<font color='red'>Question 7:</font> What is the most important feature, according to the Random Forest model?**

In [45]:
best_forrest.feature_importances_

array([0.05997873, 0.12381824, 0.06689157, 0.07133098, 0.07611842,
       0.10297833, 0.07336429, 0.09390434, 0.06891386, 0.05929251,
       0.20340873])

In [47]:
lasso1_coef = pd.DataFrame({'name': X_train.columns, 'importance':best_forrest.feature_importances_})
lasso1_coef.sort_values('importance', ascending=False)

,name,importance
10,alcohol,0.203409
1,volatile acidity,0.123818
5,free sulfur dioxide,0.102978
7,density,0.093904
4,chlorides,0.076118
6,total sulfur dioxide,0.073364
3,residual sugar,0.071331
8,pH,0.068914
2,citric acid,0.066892
0,fixed acidity,0.059979


**Make conclusions about the perdormance of the explored 3 models in this particular prediction task.**